In [ ]:
pip install selenium pandas webdriver-manager openpyxl

In [ ]:
# 임시 완성본
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
from time import sleep
from food_list import food_set
import re

def page_down(driver, num):
    store_list_area = driver.find_element(By.CSS_SELECTOR, ".Ryr1F")
    for i in range(num):
        driver.execute_script(
            "arguments[0].scrollTop += arguments[0].offsetHeight;", store_list_area
        )
        sleep(0.5)

def get_operating_hours(driver):
    operating_hours = {}
    try:
        hours_button = driver.find_element(By.CSS_SELECTOR, "a.gKP9i.RMgN0")
        if hours_button.is_displayed():
            hours_button.click()
            sleep(1)

            days = driver.find_elements(By.CSS_SELECTOR, "span.i8cJw")
            hours = driver.find_elements(By.CSS_SELECTOR, "div.H3ua4")

            for day, hour in zip(days, hours):
                operating_hours[day.text] = hour.text.strip() or None
        else:
            print("영업시간 정보가 없습니다.")
            return None

    except NoSuchElementException:
        print("영업시간 버튼을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"영업시간 정보를 가져오는 중 오류 발생: {e}")
        return None

    return operating_hours if operating_hours else None

food_dict = {
    "food_name": [],
    "address": [],
    "latitude": [],
    "longitude": [],
    "operating_hours": [],
    "phone_number": [],
    "website": [],
}

driver = wb.Chrome()
driver.get("https://map.naver.com/")

sleep(3)

search = driver.find_element(By.CSS_SELECTOR, ".input_search")
search.send_keys("호서대 음식점")
search.send_keys(Keys.ENTER)

sleep(3)

store_count = 0
page_num = 1
last_page_reached = False

while not last_page_reached:
    try:
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")
        page_down(driver, 40)
        sleep(3)

        store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
        title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")

        for data in range(len(store_list)):
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame("searchIframe")
                sleep(1)

                store_list = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")
                title = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")
                title[data].click()
                sleep(2)

                driver.switch_to.default_content()
                driver.switch_to.frame("entryIframe")
                sleep(2)

                name = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".GHAhO"))
                ).text
                
                try:
                    address = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text
                except NoSuchElementException:
                    address = None
                
                script_content = driver.find_element(By.XPATH, '//script[contains(., "window.__APOLLO_STATE__")]').get_attribute('innerHTML')
                match = re.search(r'"x":"([\d.]+)","y":"([\d.]+)"', script_content)
                if match:
                    longitude = match.group(1)
                    latitude = match.group(2)
                else:
                    longitude, latitude = None, None

                if any(food in name for food in food_set):
                    operating_hours = get_operating_hours(driver)
                    
                    try:
                        phone_number = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q").text
                    except NoSuchElementException:
                        phone_number = None
                        
                    try:
                        website = driver.find_element(By.CSS_SELECTOR, "a.CHmqa").get_attribute("href")
                    except NoSuchElementException:
                        website = None
                        
                    food_dict["food_name"].append(name)
                    food_dict["address"].append(address)
                    food_dict["latitude"].append(latitude)
                    food_dict["longitude"].append(longitude)
                    food_dict["operating_hours"].append(operating_hours)
                    food_dict["phone_number"].append(phone_number)
                    food_dict["website"].append(website)
                    
                    store_count += 1
                    print(f"{store_count}번째 음식점 정보 크롤링 완료: {name}")
                    
            except StaleElementReferenceException:
                print("페이지가 새로고침되었습니다. 다음 항목으로 넘어갑니다.")
                continue
            except Exception as e:
                print(f"데이터 추출 중 오류 발생: {e}")
                continue

        # 다음 페이지로 이동
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")
        next_btn = driver.find_elements(By.CSS_SELECTOR, ".zRM9F> a")

        if len(next_btn) > 1 and next_btn[-1].is_enabled():
            current_page = driver.find_element(By.CSS_SELECTOR, "a.qxokY").text
            next_btn[-1].click()
            sleep(2)
            
            # 페이지 변경 확인
            WebDriverWait(driver, 10).until(
                lambda d: d.find_element(By.CSS_SELECTOR, "a.qxokY").text != current_page
            )
            
            page_num += 1
            print(f"페이지 {page_num}로 이동")
        else:
            print("마지막 페이지에 도달했습니다.")
            last_page_reached = True

    except Exception as e:
        print(f"페이지 이동 중 오류 발생: {e}")
        break

driver.quit()

df = pd.DataFrame(food_dict)
df = df.fillna("NULL")

df.to_excel("food_data_stores.xlsx", index=False)

print(f"크롤링 완료 및 엑셀 파일 저장. 총 {store_count}개의 음식점 정보가 수집되었습니다.")